### 논문
- https://arxiv.org/pdf/2308.15928

In [22]:
def get_pi(arr):
    pi = [i for i in range(len(arr) + 2)]
    pi_inv = [i for i in range(len(arr) + 2)]
    for i in range(len(arr)):
        pi[i + 1] = arr[i]
        pi_inv[abs(arr[i])] = i + 1
    return pi, pi_inv

In [23]:
# def perform_reversal(pi, pi_inv, l, r, stack=[]):
#     for i in range(l, r + 1):
#         stack.append(pi[i])
#     for i in range(l, r + 1):
#         t = stack.pop()
#         pi[i] = -t
#         pi_inv[abs(t)] = i

In [24]:
def get_ov_graph(pi, pi_inv):
    spi = [0] * (2*len(pi) - 2)
    spi_inv = [0] * (2*len(pi) - 2)
    for i, num in enumerate(pi):
        if i == 0:
            spi[i] = num
            spi_inv[num] = i
        elif i == len(pi) - 1:
            spi[2*i - 1] = -num
            spi_inv[-num] = 2*i - 1
        else:
            spi[2*i - 1] = -num
            spi_inv[-num] = 2*i - 1
            spi[2*i] = num
            spi_inv[num] = 2*i
            
    ov_graph = [set([i]) for i in range(len(pi) - 1)]
    is_black = []
    for i in range(len(pi) - 1):
        pi_0 = pi[pi_inv[i]]
        if pi_0 == 0:
            pi_0 = 1
        pi_1 = pi[pi_inv[i + 1]]
        is_black.append(pi_0 * pi_1 < 0)
    
    # start of edge / end of edge
    soe = [None for i in range(2*len(pi) - 2)]
    eoe = [None for i in range(2*len(pi) - 2)]
    
    for i in range(len(pi) - 1):
        # v_i
        s = spi_inv[i]
        e = spi_inv[-(i + 1)]
        soe[min(s, e)] = i
        eoe[max(s, e)] = i
    
    stack1 = []
    stack2 = []
    for i in range(2*len(pi) - 2):
        if soe[i] is not None:
            stack1.append(soe[i])
        if eoe[i] is not None:
            while stack1[-1] != eoe[i]:
                overlapping = stack1.pop()
                stack2.append(overlapping)
                ov_graph[eoe[i]].add(overlapping)
                ov_graph[overlapping].add(eoe[i])
            stack1.pop()
            while stack2:
                stack1.append(stack2.pop())

    return ov_graph, is_black

In [25]:
from collections import deque


def get_nsawcc(ov_graph, is_black):
    # Non-Singleton All-White Connected Component
    queue = deque()
    visited = [False] * len(is_black)
    for i in range(len(is_black)):
        if visited[i]:
            continue
        visited[i] = True
        queue.append(i)
        is_all_white = not is_black[i]
        count = 1
        while queue:
            cur = queue.popleft()
            for next_ in ov_graph[cur]:
                if visited[next_]:
                    continue
                visited[next_] = True
                count += 1
                if is_black[next_]:
                    is_all_white = False
                queue.append(next_)
        if count > 1 and is_all_white:
            print('detected nsawcc containing v{i}')
            return i

In [26]:
def toggle(ov_graph, is_black, vertex):
    affected = set(ov_graph[vertex])
    for v in affected:
        ov_graph[v] ^= affected
        ov_graph[v] |= {v}
        is_black[v] = not is_black[v]

In [27]:
BLUE_PREV = 0
BLUE_NEXT = 1
RED_PREV = 2
RED_NEXT = 3

def get_br_graph(pi, pi_inv):
    br_graph = {}
    
    for i in range(len(pi)):
        pos_node = [None, None, None, None]
        neg_node = [None, None, None, None]

        abs_value = abs(pi[i])
        br_graph[f'+{abs_value}'] = pos_node
        br_graph[f'-{abs_value}'] = neg_node
        
        if i > 0:
            abs_prev = abs(pi[i - 1]) 
            pos_node[BLUE_PREV] = f'+{abs_prev}'
            neg_node[BLUE_PREV] = f'-{abs_prev}'
            
            sign = pi[i] >= 0
            prev_inv = pi[pi_inv[abs(pi[i]) - 1]]
            prev_sign = prev_inv >= 0
            abs_prev_inv = abs(prev_inv)
            if sign == prev_sign:
                pos_node[RED_PREV] = f'+{abs_prev_inv}'
                neg_node[RED_PREV] = f'-{abs_prev_inv}'
            else:
                pos_node[RED_PREV] = f'-{abs_prev_inv}'
                neg_node[RED_PREV] = f'+{abs_prev_inv}'
        if i < len(pi) - 1:
            abs_next = abs(pi[i + 1]) 
            pos_node[BLUE_NEXT] = f'+{abs_next}'
            neg_node[BLUE_NEXT] = f'-{abs_next}'
            
            sign = pi[i] >= 0
            next_inv = pi[pi_inv[abs(pi[i]) + 1]]
            next_sign = next_inv >= 0
            abs_next_inv = abs(next_inv)
            if sign == next_sign:
                pos_node[RED_NEXT] = f'+{abs_next_inv}'
                neg_node[RED_NEXT] = f'-{abs_next_inv}'
            else:
                pos_node[RED_NEXT] = f'-{abs_next_inv}'
                neg_node[RED_NEXT] = f'+{abs_next_inv}'
    return br_graph

In [20]:
def perform_reversal(br_graph, start, end):
    pos_cur = '+0'
    neg_cur = '-0'
    
    pos_prefix = None
    neg_prefix = None 
    
    pos_start = None
    neg_start = None
    
    for i in range(len(br_graph) // 2):
        if i == start - 1:
            pos_prefix = pos_cur
            neg_prefix = neg_cur
            
        elif start <= i <= end:
            pos_next = br_graph[pos_cur][BLUE_NEXT]
            neg_next = br_graph[neg_cur][BLUE_NEXT]
            
            br_graph[pos_cur][BLUE_NEXT], br_graph[pos_cur][BLUE_PREV] = br_graph[pos_cur][BLUE_PREV], br_graph[pos_cur][BLUE_NEXT]
            br_graph[neg_cur][BLUE_NEXT], br_graph[neg_cur][BLUE_PREV] = br_graph[neg_cur][BLUE_PREV], br_graph[neg_cur][BLUE_NEXT]
            
            if i == start:
                pos_start = pos_cur
                neg_start = neg_cur
            elif i == end:
                br_graph[pos_cur][BLUE_PREV] = neg_prefix
                br_graph[neg_cur][BLUE_PREV] = pos_prefix
                br_graph[pos_prefix][BLUE_NEXT] = neg_cur
                br_graph[neg_prefix][BLUE_NEXT] = pos_cur
            
            pos_cur = pos_next
            neg_cur = neg_next
            continue
        
        elif i == end + 1:
            br_graph[pos_cur][BLUE_PREV] = neg_start
            br_graph[neg_cur][BLUE_PREV] = pos_start
            
            br_graph[pos_start][BLUE_NEXT] = neg_cur
            br_graph[neg_start][BLUE_NEXT] = pos_cur
            break
        
        pos_cur = br_graph[pos_cur][BLUE_NEXT]
        neg_cur = br_graph[neg_cur][BLUE_NEXT]

In [21]:
arr = [-2, -5, 1, -4, 6, -3]
pi, pi_inv = get_pi(arr)
print(pi)
br_graph = get_br_graph(pi, pi_inv)
display(br_graph)

perform_reversal(br_graph, 2, 4)
br_graph

[0, -2, -5, 1, -4, 6, -3, 7]


{'+0': [None, '+2', None, '+1'],
 '-0': [None, '-2', None, '-1'],
 '+2': ['+0', '+5', '-1', '+3'],
 '-2': ['-0', '-5', '+1', '-3'],
 '+5': ['+2', '+1', '+4', '-6'],
 '-5': ['-2', '-1', '-4', '+6'],
 '+1': ['+5', '+4', '+0', '-2'],
 '-1': ['-5', '-4', '-0', '+2'],
 '+4': ['+1', '+6', '+3', '+5'],
 '-4': ['-1', '-6', '-3', '-5'],
 '+6': ['+4', '+3', '-5', '+7'],
 '-6': ['-4', '-3', '+5', '-7'],
 '+3': ['+6', '+7', '+2', '+4'],
 '-3': ['-6', '-7', '-2', '-4'],
 '+7': ['+3', None, '+6', None],
 '-7': ['-3', None, '-6', None]}

{'+0': [None, '+2', None, '+1'],
 '-0': [None, '-2', None, '-1'],
 '+2': ['+0', '-4', '-1', '+3'],
 '-2': ['-0', '+4', '+1', '-3'],
 '+5': ['+1', '-6', '+4', '-6'],
 '-5': ['-1', '+6', '-4', '+6'],
 '+1': ['+4', '+5', '+0', '-2'],
 '-1': ['-4', '-5', '-0', '+2'],
 '+4': ['-2', '+1', '+3', '+5'],
 '-4': ['+2', '-1', '-3', '-5'],
 '+6': ['-5', '+3', '-5', '+7'],
 '-6': ['+5', '-3', '+5', '-7'],
 '+3': ['+6', '+7', '+2', '+4'],
 '-3': ['-6', '-7', '-2', '-4'],
 '+7': ['+3', None, '+6', None],
 '-7': ['-3', None, '-6', None]}

# try1

### 입력

In [48]:
arr = [-3, 1, -2, 6, 4, 5 , 8, 7]

### reverse 정의

In [50]:
def perform_reversal(arr, i, j, arr_inv):
    reversed = []
    for k in range(i, j + 1):
        reversed.append(-arr[k])
    for k in range(i, j + 1):
        value = reversed.pop()
        arr[k] = value
        arr_inv[abs(value)] = k

### 전처리

In [51]:
pi = [0] + arr + [len(arr) + 1]
n = pi[-1]
signed_pi_inv = [0] * (2*n)
pi_inv = [0] * (n + 1)
signed_pi = [0]
signed_pi_inv[0] = 0
pi_inv[0] = 0
pi_inv[n] = n
for i, num in enumerate(arr, 1):
    signed_pi_inv[-num] = len(signed_pi)
    signed_pi.append(-num)
    signed_pi_inv[num] = len(signed_pi)
    signed_pi.append(num)
    pi_inv[abs(num)] = i
signed_pi_inv[-n] = len(signed_pi)
signed_pi.append(-n)

edge_start_at = [None] * (2*n)
edge_end_at = [None] * (2*n)
black = [True] * n
for i in range(0, n):
    s = signed_pi_inv[i]
    e = signed_pi_inv[-(i + 1)]
    if s > e:
        s, e = e, s
    edge_start_at[s] = i
    edge_end_at[e] = i
    black[i] = ((1 if pi[i] >= 0 else -1) * pi[i + 1]) < 0

G = {i: {i} for i in range(n)}
S1 = []
S2 = []
for i in range(0, 2*n):
    if (v := edge_start_at[i]) != None:
        S1.append(v)
    elif (v := edge_end_at[i]) != None:
        while S1[-1] != v:
            S2.append(s := S1.pop())
            G[v].add(s)
            G[s].add(v)
        S1.pop()
        while S2:
            S1.append(S2.pop())

In [52]:
# non-singleton all-white connected component
from collections import deque

reversals = []
while True:
    nsawcc = []
    visited = [False] * n
    for i in range(n):
        if visited[i]:
            continue
        visited[i] = True
        connected = 0
        is_all_white = True
        queue = deque([i])
        while queue:
            here = queue.popleft()
            if black[here]:
                is_all_white = False
                break
            connected += 1
            for there in G[here]:
                if there == here or visited[there]:
                    continue
                visited[there] = True
                queue.append(there)
        if is_all_white and connected > 1:
            print(f'{i} is an element of non-singleton all-white connected component')
            if i == 0:
                nsawcc.append((pi_inv[i+1], pi_inv[i+1]))
            else:
                nsawcc.append((pi_inv[i], pi_inv[i]))
    if not nsawcc:
        break
    
    reversals.extend(nsawcc)
    for l, r in nsawcc:
        print(f'reversing [{l}, {r}]')
        perform_reversal(pi, l, r, pi_inv)
        print(pi)
        print(pi_inv)

    signed_pi_inv = [0] * (2*n)
    pi_inv = [0] * (n + 1)
    signed_pi = [0]
    signed_pi_inv[0] = 0
    pi_inv[0] = 0
    pi_inv[n] = n
    for i, num in enumerate(arr, 1):
        signed_pi_inv[-num] = len(signed_pi)
        signed_pi.append(-num)
        signed_pi_inv[num] = len(signed_pi)
        signed_pi.append(num)
        pi_inv[abs(num)] = i
    signed_pi_inv[-n] = len(signed_pi)
    signed_pi.append(-n)

    edge_start_at = [None] * (2*n)
    edge_end_at = [None] * (2*n)
    black = [True] * n
    for i in range(0, n):
        s = signed_pi_inv[i]
        e = signed_pi_inv[-(i + 1)]
        if s > e:
            s, e = e, s
        edge_start_at[s] = i
        edge_end_at[e] = i
        black[i] = ((1 if pi[i] >= 0 else -1) * pi[i + 1]) < 0

    G = {i: {i} for i in range(n)}
    S1 = []
    S2 = []
    for i in range(0, 2*n):
        if (v := edge_start_at[i]) != None:
            S1.append(v)
        elif (v := edge_end_at[i]) != None:
            while S1[-1] != v:
                S2.append(s := S1.pop())
                G[v].add(s)
                G[s].add(v)
            S1.pop()
            while S2:
                S1.append(S2.pop())

5 is an element of non-singleton all-white connected component
reversing [6, 6]
[0, -3, 1, -2, 6, 4, -5, 8, 7, 9]
[0, 2, 3, 1, 5, 6, 4, 8, 7, 9]
7 is an element of non-singleton all-white connected component
reversing [8, 8]
[0, -3, 1, -2, 6, 4, -5, 8, -7, 9]
[0, 2, 3, 1, 5, 6, 4, 8, 7, 9]


In [53]:
print(pi)
print(pi_inv)
print(signed_pi)
print(signed_pi_inv)
G

[0, -3, 1, -2, 6, 4, -5, 8, -7, 9]
[0, 2, 3, 1, 5, 6, 4, 8, 7, 9]
[0, 3, -3, -1, 1, 2, -2, -6, 6, -4, 4, -5, 5, -8, 8, -7, 7, -9]
[0, 4, 5, 1, 10, 12, 8, 16, 14, 17, 13, 15, 7, 11, 9, 2, 6, 3]


{0: {0, 2, 3},
 1: {1, 2},
 2: {0, 1, 2},
 3: {0, 3, 5, 6},
 4: {4},
 5: {3, 5, 6},
 6: {3, 5, 6, 7, 8},
 7: {6, 7, 8},
 8: {6, 7, 8}}

### toggle 정의

In [54]:
def toggle(vertices, G, black):
    print('@toggle START')
    print(f'\t{G=}')
    print(f'\t{black=}')
    ret = set(vertices)
    for neighbor in vertices:
        G[neighbor] = G[neighbor] ^ vertices | {neighbor}
        black[neighbor] = not black[neighbor]
    print(f'>>>\n\t{G=}')
    print(f'\t{black=}')
    print('@toggle END')
    return ret

### reversal_sort 정의

In [55]:
def reversal_sort(G, black):
    S1, S2 = [], []
    V = set()
    
    for v in G:
        if len(G[v]) != 1 or black[v]:
            V.add(v)
    
    print(f'initial {V=}\n')
            
    def toggle_black():
        for v in V:
            if black[v]:
                break
        affected = toggle(G[v], G, black)
        for v in set(V):
            if len(G[v]) == 1 and not black[v]:
                V.remove(v)
        S1.append((v, affected))
        print(f'=== toggled, {V=}\n')

    def undo_last_toggle():
        w, to_undo = S1.pop()
        affected = toggle(to_undo, G, black)
        print(f'=== undone toggle, {V=}\n')
        return w, affected

    while any(black[v] for v in V):
        toggle_black()
    
    while V:
        while all(not black[v] for v in V):
            undone = undo_last_toggle()
            S2.append(undone)
        while any(black[v] for v in V):
            toggle_black()
        if not black[S2[-1][0]]:
            undo_last_toggle()
        
    print()
    print(f'{S1=}')
    print(f'{S2=}')
    print(f'{V=}')
    print(f'{G=}')
    print(f'{black=}')

    ret = []
    for sort in S1 + S2[::-1]:
        ret.append(sort[0])
    return ret

### 테스트

In [56]:
ov_reversals = reversal_sort(G, black)
print(ov_reversals)

initial V={0, 1, 2, 3, 5, 6, 7, 8}

@toggle START
	G={0: {0, 2, 3}, 1: {1, 2}, 2: {0, 1, 2}, 3: {0, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[True, True, True, True, False, True, True, True, True]
>>>
	G={0: {0}, 1: {1, 2}, 2: {1, 2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[False, True, False, False, False, True, True, True, True]
@toggle END
=== toggled, V={1, 2, 3, 5, 6, 7, 8}

@toggle START
	G={0: {0}, 1: {1, 2}, 2: {1, 2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[False, True, False, False, False, True, True, True, True]
>>>
	G={0: {0}, 1: {1}, 2: {2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {3, 5, 6}, 6: {3, 5, 6, 7, 8}, 7: {8, 6, 7}, 8: {8, 6, 7}}
	black=[False, False, True, False, False, True, True, True, True]
@toggle END
=== toggled, V={2, 3, 5, 6, 7, 8}

@toggle START
	G={0: {0}, 1: {1}, 2: {2, 3}, 3: {2, 3, 5, 6}, 4: {4}, 5: {

In [59]:
sorted_pi = list(pi)
sorted_pi_inv = list(pi_inv)
print(sorted_pi)
for v in ov_reversals:
    a = min(sorted_pi_inv[v], sorted_pi_inv[v + 1])
    b = max(sorted_pi_inv[v], sorted_pi_inv[v + 1])
    if pi[a] + pi[b] == 1:
        b -= 1
    else:
        a += 1
    perform_reversal(sorted_pi, a, b, sorted_pi_inv)
    print(a, b, sorted_pi, sorted_pi_inv)

[0, -3, 1, -2, 6, 4, -5, 8, -7, 9]
8 9 [0, -3, 1, -2, 6, 4, -5, 8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
7 7 [0, -3, 1, -2, 6, 4, -5, -8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
7 7 [0, -3, 1, -2, 6, 4, -5, 8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
7 7 [0, -3, 1, -2, 6, 4, -5, -8, -9, 7] [0, 2, 3, 1, 5, 6, 4, 9, 7, 8]
8 9 [0, -3, 1, -2, 6, 4, -5, -8, -7, 9] [0, 2, 3, 1, 5, 6, 4, 8, 7, 9]
5 8 [0, -3, 1, -2, 6, 7, 8, 5, -4, 9] [0, 2, 3, 1, 8, 7, 4, 5, 6, 9]


: 